# 💹 Project: Finance. Forecasting Currency Exchange Rates for Banking Operations

This project focuses on forecasting **SGD/USD** and **CNY/USD** exchange rates to support banking operations. We’ll compare **ARIMA** (statistical baseline), **XGBoost** (feature-based ML), **LSTM** (deep learning sequence model), and **TFT** (Transformer-based deep learning model) for forecasting accuracy.

## Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler as DartsScaler
from darts.metrics import rmse, mae, mape
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.models import ARIMA, XGBModel, RNNModel, TFTModel

from darts.utils.model_selection import train_test_split
from darts.utils.likelihood_models import QuantileRegression

import warnings
warnings.filterwarnings("ignore")
import warnings

## Time Series Analysis Helpers

In [ ]:
plt.rcParams["figure.figsize"] = (11, 5)

def adf_test(y, name="series"):
    res = adfuller(y, autolag="AIC")
    print(f"ADF for {name}: stat={res[0]:.3f}, p={res[1]:.4f}")

def kpss_test(y, name="series"):
    try:
        stat, p, lags, crit = kpss(y, regression='c', nlags='auto')
        print(f"KPSS for {name}: stat={stat:.3f}, p={p:.4f}")
    except Exception as e:
        print("KPSS error:", e)

def difference(series: pd.Series, d: int = 1) -> pd.Series:
    s = series.copy()
    for _ in range(d):
        s = s.diff()
    return s.dropna()

## Import and Manipulate Data

In [ ]:
df = pd.read_csv("Foreign_Exchange_Rates.csv", parse_dates=["DATE"])
df = df.set_index("DATE").sort_index()

df = df.replace('ND', np.nan)

df = df.rename(columns={
    "SINGAPORE - SINGAPORE DOLLAR/US$": "SGDUSD",
    "CHINA - YUAN/US$": "CNYUSD"
})

df['SGDUSD'] = pd.to_numeric(df['SGDUSD'], errors='coerce')
df['CNYUSD'] = pd.to_numeric(df['CNYUSD'], errors='coerce')

df = df.ffill().bfill()

display(df.head())
df[['SGDUSD','CNYUSD']].plot(title="SGD/USD and CNY/USD")
plt.show()

- Long-run trends

  - CNY/USD: Flat near 8.3 until ~2005 (hard peg), then a long appreciation to ~6.1 by 2014, followed by a regime break/depreciation toward ~6.9–7.1 in 2015–2019.

 - SGD/USD: Gradual appreciation from 1.75 (early 2000s) to 1.23 (2011–2012), then modest depreciation and range-bound behavior (~1.32–1.40) mid-2014 onward.

- Volatility & structure

  - CNY shows structural breaks (2005 de-peg; 2015 devaluation; 2018 trade tensions) → higher regime-driven volatility.

  - SGD path is smoother/lower vol (consistent with MAS’s band-managed policy).

- Co-movement

  - Limited before 2005 (CNY peg).

  - Noticeable joint weakening vs USD in 2014–2016, then partial recoveries/divergence.

- Seasonality

  - No clear seasonality by eye; verify with ACF/PACF and STL decompositio

## Time Series EDA

In [ ]:
for col in ["SGDUSD", "CNYUSD"]:
    print("\n====", col, "====")
    y = df[col].astype(float).dropna()
    adf_test(y, col)
    kpss_test(y, col)

for col in ["SGDUSD", "CNYUSD"]:
    print("\n---- ACF/PACF (possibly differenced) for", col, "----")
    y = df[col].astype(float).dropna()
    needs_diff = (adfuller(y)[1] > 0.05)
    if needs_diff:
        y_used = difference(y, 1)
        d = 1
    else:
        y_used = y
        d = 0
    print(f"Using d={d}")

    fig, axes = plt.subplots(1, 2, figsize=(12,4))
    plot_acf(y_used, ax=axes[0], lags=40, title=f"{col} - ACF (d={d})")
    plot_pacf(y_used, ax=axes[1], lags=40, title=f"{col} - PACF (d={d})", method="ywm")
    plt.show()

**Stationarity & ACF/PACF Diagnostics**

- Tests on raw levels

- SGDUSD — ADF p = 0.726 (fail to reject unit root), KPSS p = 0.010 (reject stationarity) → non-stationary.

- CNYUSD — ADF p = 0.636, KPSS p = 0.010 → non-stationary.

- Implication: both series need first differencing (or log-returns). This motivates d = 1 in ARIMA.

**ACF/PACF on first differences (d = 1)**

- SGDUSD (d=1): ACF and PACF show no materially significant spikes at low lags → differenced series ≈ white noise.

- CNYUSD (d=1): Same picture—no clear AR or MA structure.

- Seasonality: No visible weekly/monthly seasonal pattern in ACF; seasonal terms likely unnecessary.

- Takeaway: When the differenced series is close to white noise, the data behave like a random walk. A strong, simple baseline is ARIMA(0,1,0) (random walk), optionally with drift.

## Create Time Series, Covariates, and Splits

In [ ]:
sgd_series = TimeSeries.from_series(df["SGDUSD"].astype(np.float32))
cny_series = TimeSeries.from_series(df["CNYUSD"].astype(np.float32))

day_of_week = datetime_attribute_timeseries(df.index, attribute="day_of_week", one_hot=True).astype(np.float32)
month = datetime_attribute_timeseries(df.index, attribute="month", one_hot=True).astype(np.float32)
calendar_covs = day_of_week.stack(month).astype(np.float32)

sgd_train, sgd_val = train_test_split(sgd_series, test_size=0.2)
cny_train, cny_val = train_test_split(cny_series, test_size=0.2)

scaler_target = DartsScaler()
sgd_train_s = scaler_target.fit_transform(sgd_train)
sgd_val_s = scaler_target.transform(sgd_val)

scaler_covs = DartsScaler()
calendar_covs_s = scaler_covs.fit_transform(calendar_covs)
cov_train, cov_val = calendar_covs_s.split_after(sgd_train.end_time())

## ARIMA Model Training and Evaluation

### Sinapore Dollar

In [ ]:
h = len(sgd_val)

naive_pred = TimeSeries.from_times_and_values(sgd_val.time_index, sgd_train[-1].values()[0] * np.ones(h))
naive_rmse = rmse(sgd_val, naive_pred)
print("Naïve (last value) RMSE:", naive_rmse)

# ARIMA: choose (p,d,q) using your ACF/PACF inspection
p, d, q = 0, 1, 0

arima = ARIMA(p, d, q, trend='t')
arima.fit(sgd_train)
arima_pred = arima.predict(h)
print("ARIMA RMSE:", rmse(sgd_val, arima_pred))

plt.figure(figsize=(12, 6))
sgd_val.plot(label="Actual SGD")
naive_pred.plot(label="Naive (Last Value) Prediction SGD")
arima_pred.plot(label="ARIMA Prediction SGD") # Corrected variable name
plt.title("Naive vs ARIMA Predictions for SGD/USD")
plt.legend()
plt.show()


📉 Naïve vs. ARIMA(0,1,0) with drift — SGD/*USD*

Visual takeaways

- The naïve forecast badly overestimates once the series trends down (flat line above the actuals for most of the window).

- ARIMA with drift captures the downward slope across the period, removing the persistent bias you see in the naïve line.

- Neither model captures short-term oscillations; both are deliberately smooth. The ARIMA line explains the medium-term direction; the naïve line explains nothing.

Accuracy (from the same run)

- Naïve: RMSE ≈ 0.0558, MAE ≈ 0.0503

- ARIMA w/ drift: RMSE ≈ 0.0328, MAE ≈ 0.0254
→ ~41% lower RMSE and ~49% lower MAE vs naïve, consistent with the visual improvement.

Interpretation for banking use

- Over this horizon the SGD showed a gentle appreciation vs USD (lower SGD/USD). The drift term learns that slope, yielding tighter, less biased forecasts.

- Because FX often behaves like a random walk, adding a small drift can help when a clear medium-term trend is present; otherwise it won’t.

Limitations & next steps

- The linear drift cannot capture local cycles, volatility spikes, or regime breaks.

- Run rolling backtests (e.g., 1-day and 5-day horizons).

- Check residuals (e.g., Ljung–Box) and volatility clustering; consider volatility features or GARCH if needed.

- For richer dynamics, complement with feature-based (XGB) or sequence models (LSTM/TFT) using covariates (calendar, interest-rate differentials, DXY).

### Chinese Yaun

In [ ]:
h_cny = len(cny_val)

naive_pred_cny = TimeSeries.from_times_and_values(cny_val.time_index, cny_train[-1].values()[0] * np.ones(h_cny))
naive_rmse_cny = rmse(cny_val, naive_pred_cny)
print("Naïve (last value) RMSE for CNY:", naive_rmse_cny)

p_cny, d_cny, q_cny = 1, 1, 1

arima_cny = ARIMA(p_cny, d_cny, q_cny, trend='t')
arima_cny.fit(cny_train)
arima_pred_cny = arima_cny.predict(h_cny)
print("ARIMA RMSE for CNY:", rmse(cny_val, arima_pred_cny))

plt.figure(figsize=(12, 6))
cny_val.plot(label="Actual CNY")
naive_pred_cny.plot(label="Naive (Last Value) Prediction CNY")
arima_pred_cny.plot(label="ARIMA Prediction CNY")
plt.title("Naive vs ARIMA Predictions for CNY/USD")
plt.legend()
plt.show()

**Interpretation**

📉 Naïve vs. ARIMA(0,1,0)+drift — CNY/USD (validation window)

Visual diagnosis

- From 2016 onward the series rises toward ~7.0, with a dip in early-2018 and renewed increases in 2019.

- The naïve line underestimates most of the period (flat line below actuals) but stays relatively close early on.

- The ARIMA+drift forecast assumes a persistent decline (continued appreciation of CNY vs USD). That trend is the wrong sign for this window, so its error grows over time (systematic negative bias).

Interpretation

- CNY/USD behaves like a random walk with regime/level shifts.

- The drift term was learned from the training window (likely mild downward trend) but doesn’t hold out-of-sample; hence the linear decline misses the subsequent upward move.

- In such settings, a random walk without drift can outperform a mis-specified drift model.

## XGBoost Forecasting Model SGD/USD

In [ ]:
target_lags = [-1, -2, -3, -5, -7, -14, -21]
past_cov_lags = [-1, -2, -3, -5, -7]
future_cov_lags = [0]   #0..6, see the "extend covariates" block below

xgb = XGBModel(
    lags=target_lags,
    lags_past_covariates=past_cov_lags,
    lags_future_covariates=future_cov_lags,
    random_state=42,
    n_estimators=400,
    learning_rate=0.05,
    max_depth=6
)

scaler_cny = DartsScaler()
cny_series_s = scaler_cny.fit_transform(cny_series)
cny_train_s, cny_val_s = cny_series_s.split_after(sgd_train.end_time())
past_cov_train = cny_train_s
past_cov_full = cny_train_s.append(cny_val_s)

xgb.fit(
    series=sgd_train_s,
    past_covariates=past_cov_train,
    future_covariates=cov_train
)

xgb_pred_s = xgb.predict(
    n=h,
    past_covariates=past_cov_full,
    future_covariates=cov_val
)
xgb_pred = scaler_target.inverse_transform(xgb_pred_s)

print("XGB RMSE:", rmse(sgd_val, xgb_pred))
print("XGB MAE:",  mae(sgd_val, xgb_pred))
print("XGB MAPE:", mape(sgd_val, xgb_pred))

sgd_series.plot(label="Actual")
xgb_pred.plot(label="XGB forecast")
plt.title("SGD/USD — XGBModel")
plt.legend(); plt.show()

**Interpretation**

XGB RMSE = 0.0362, MAE = 0.0307, MAPE ≈ 2.26%

With SGD/USD ≈ 1.35, that’s ~3.1¢ average absolute error and ~2.26% relative error, respectable for daily FX.

Compared to earlier baselines:

- vs Naïve (RMSE 0.0558): ~35% improvement → strong win.

- vs ARIMA(0,1,0)+drift (RMSE 0.0328): ~10% worse → ARIMA-drift is still narrowly ahead on this split.

## XGBoost Forecasting Model CNY/USD

In [ ]:
target_lags_cny = [-1, -2, -5, -10, -20]
past_cov_lags_cny = [-1, -2, -5, -10]
future_cov_lags_cny = [0, 1, 2, 3, 4, 5, 6]

xgb_cny = XGBModel(
    lags=target_lags_cny,
    lags_past_covariates=past_cov_lags_cny,
    lags_future_covariates=future_cov_lags_cny,
    random_state=42,
    n_estimators=400,
    learning_rate=0.05,
    max_depth=6
)

scaler_target_cny = DartsScaler()
cny_train_s = scaler_target_cny.fit_transform(cny_train)
cny_val_s = scaler_target_cny.transform(cny_val)

scaler_sgd = DartsScaler()
sgd_series_s = scaler_sgd.fit_transform(sgd_series)
sgd_train_s_cov, sgd_val_s_cov = sgd_series_s.split_after(cny_train.end_time())

past_cov_train_cny = sgd_train_s_cov
past_cov_full_cny = sgd_train_s_cov.append(sgd_val_s_cov)

h_cny = len(cny_val)
max_future = max(future_cov_lags_cny)
freq = pd.infer_freq(cny_series.time_index) or "D"
offset = pd.tseries.frequencies.to_offset(freq)

extended_index = pd.date_range(
    start=cny_series.start_time(),
    end=cny_series.end_time() + (h_cny + max_future) * offset,
    freq=freq
)

dow = datetime_attribute_timeseries(extended_index, attribute="day_of_week", one_hot=True)
mon = datetime_attribute_timeseries(extended_index, attribute="month", one_hot=True)
calendar_full = dow.stack(mon)

scaler_covs = DartsScaler()
calendar_full_s = scaler_covs.fit_transform(calendar_full)

cov_train_cny, cov_future_full_cny = calendar_full_s.split_after(cny_train.end_time())

xgb_cny.fit(
    series=cny_train_s,
    past_covariates=past_cov_train_cny,
    future_covariates=cov_train_cny
)

xgb_pred_s_cny = xgb_cny.predict(
    n=h_cny,
    past_covariates=past_cov_full_cny,
    future_covariates=cov_future_full_cny
)
xgb_pred_cny = scaler_target_cny.inverse_transform(xgb_pred_s_cny)

print("XGB RMSE for CNY:", rmse(cny_val, xgb_pred_cny))
print("XGB MAE  for CNY:", mae(cny_val, xgb_pred_cny))
print("XGB MAPE for CNY:", mape(cny_val, xgb_pred_cny))

cny_series.plot(label="Actual CNY")
xgb_pred_cny.plot(label="XGB forecast CNY")
plt.title("CNY/USD — XGBModel (with SGD as past covariate)")
plt.legend(); plt.show()

📈 CNY/USD — XGB with SGD as past covariate (validation)
What the plot shows

The blue XGB forecast is too flat and lags the upward moves in 2018–2019.

This indicates underfitting / regime mismatch: the model learned a stable level near 6.45–6.75 and didn’t adapt to the later uptrend (~6.9–7.1).

Metrics

- RMSE = 0.2413, MAE = 0.2017, MAPE ≈ 2.99%.

- With CNY/USD ≈ 6.6–7.0, that’s about ¥0.20 average absolute error per USD (~3%).

- Use case:

  - Daily pricing/hedging → 3% is too high.

 - Longer-horizon planning → may be acceptable, but can improve.

Why performance lags

- Regimes: Training likely spans pre-2005 peg and 2015–2019 policy shifts; that older history can dilute signal for recent dynamics.

- Lags: Weekly calendar lags (-7,-14,-21) are less relevant on business-day FX data.

- Target in levels: Modeling levels makes it harder to catch directional shifts; returns often work better.

- Covariates: Only SGD as past cov; no macro/future-known features beyond calendar.

## LSTM Forecasting with Calendar Features (CNY/USD)

In [ ]:
lstm_cny = RNNModel(
    model="LSTM",
    input_chunk_length=24,
    hidden_dim=64,
    n_rnn_layers=2,
    dropout=0.1,
    n_epochs=50,
    random_state=42,
)

assert cov_train.end_time() >= cny_train_s.end_time(), \
    "cov_train must extend through (or beyond) the end of the training period."

cov_full = cov_train.append(cov_val)

step = cny_train_s.time_index[1] - cny_train_s.time_index[0]
required_start = cny_train_s.end_time() - (lstm_cny.input_chunk_length - 1) * step

assert cov_full.start_time() <= required_start, \
    f"future_covariates must start on/before {required_start} to cover the lookback window."
assert cov_full.end_time() >= cny_val.end_time(), \
    "future_covariates must extend through the forecast horizon."

lstm_cny.fit(
    series=cny_train_s,
    future_covariates=cov_train,
    verbose=True
)

h_cny = len(cny_val)
lstm_pred_s_cny = lstm_cny.predict(
    n=h_cny,
    future_covariates=cov_full
)

scaler_target_cny = DartsScaler()
scaler_target_cny.fit(cny_train)
lstm_pred_cny = scaler_target_cny.inverse_transform(lstm_pred_s_cny)

print("LSTM RMSE for CNY:", rmse(cny_val, lstm_pred_cny))
print("LSTM MAE for CNY:",  mae(cny_val, lstm_pred_cny))

ax = cny_series.plot(label="Actual CNY")
lstm_pred_cny.plot(ax=ax, label="LSTM forecast CNY")
plt.title("CNY/USD — LSTM (DARTS RNNModel, future covariates only)")
plt.legend(); plt.show()

**Interpretation**

The model is underfitting and exhibits regression-to-the-mean:

- The model trained with input_chunk_length=24 (short memory) and a small LSTM. It mostly extrapolates the longer 2010–2015 downtrend it saw rather than reacting to the later regime change.

- In this setup RNNModel only uses future_covariates. If those are calendar features, they carry little predictive signal for FX; the network effectively becomes a smoothed random walk.

- output_chunk_length is forced to 1 for RNNModel, so the forecast is rolled one step at a time—errors accumulate and the line looks flat/smoothed.

The picture is consistent with the earlier metrics you reported for this LSTM (e.g., higher RMSE/MAE than XGB and naïve RW), i.e., no lift over baselines.

## LSTM Forecasting with Calendar Features (SGD/USD)

In [ ]:
lstm_sgd = RNNModel(
    model="LSTM",
    input_chunk_length=24,
    hidden_dim=64,
    n_rnn_layers=2,
    dropout=0.1,
    n_epochs=50,
    random_state=42,
)

assert cov_train.end_time() >= sgd_train_s.end_time(), \
    "cov_train must extend through (or beyond) the end of the training period."

cov_full = cov_train.append(cov_val)

step = sgd_train_s.time_index[1] - sgd_train_s.time_index[0]
required_start = sgd_train_s.end_time() - (lstm_sgd.input_chunk_length - 1) * step

assert cov_full.start_time() <= required_start, \
    f"future_covariates must start on/before {required_start} to cover the lookback window."
assert cov_full.end_time() >= sgd_val.end_time(), \
    "future_covariates must extend through the forecast horizon."

lstm_sgd.fit(
    series=sgd_train_s,
    future_covariates=cov_train,
    verbose=True
)

lstm_pred_s = lstm_sgd.predict(
    n=h,
    future_covariates=cov_full
)

lstm_pred = scaler_target.inverse_transform(lstm_pred_s)

print("LSTM RMSE:", rmse(sgd_val, lstm_pred))
print("LSTM MAE:",  mae(sgd_val, lstm_pred))

ax = sgd_series.plot(label="Actual")
lstm_pred.plot(ax=ax, label="LSTM forecast")
plt.title("SGD/USD — LSTM (DARTS RNNModel, future covariates only)")
plt.legend(); plt.show()

**Interpretation**

- Inverse-scaling bug, the forecast was de-standardized with the wrong scaler (e.g., a scaler fit on CNY or on the full series instead of the SGD train scaler instance). That would shift & stretch predictions to the wrong magnitude (explains ~1.9).

- Mismatched covariates or time alignment, future_covariates do not cover the required lookback/horizon or are misaligned with the target index, making the net extrapolate oddly.

- Target/covariate swap, accidentally passing a scaled SGD series (or another level) as a future covariate; LSTM may latch onto it in a way that explodes when rolled forward.

## Global Forecasting with TFT Model (SGD/USD & CNY/USD)

In [ ]:
sgd_train, sgd_val = train_test_split(sgd_series, test_size=0.2)
cny_train, cny_val = train_test_split(cny_series, test_size=0.2)

scaler_sgd = DartsScaler()
sgd_train_s = scaler_sgd.fit_transform(sgd_train)
sgd_val_s = scaler_sgd.transform(sgd_val)

scaler_cny = DartsScaler()
cny_train_s = scaler_cny.fit_transform(cny_train)
cny_val_s = scaler_cny.transform(cny_val)

h_sgd = len(sgd_val)
h_cny = len(cny_val)
h_max = max(h_sgd, h_cny)

start_idx = min(sgd_train.start_time(), cny_train.start_time())
end_idx = max(sgd_train.end_time(), cny_train.end_time())

freq = pd.infer_freq(sgd_series.time_index) or pd.infer_freq(cny_series.time_index) or "D"
offset = pd.tseries.frequencies.to_offset(freq)

extended_index = pd.date_range(start=start_idx, end=end_idx + h_max * offset, freq=freq)

dow = datetime_attribute_timeseries(extended_index, attribute="day_of_week", one_hot=True)
mon = datetime_attribute_timeseries(extended_index, attribute="month", one_hot=True)
calendar_full = dow.stack(mon)

scaler_cov = DartsScaler()
calendar_full_s = scaler_cov.fit_transform(calendar_full).astype(np.float32)

cov_train_sgd = calendar_full_s.slice(start_idx, sgd_train.end_time())
cov_train_cny = calendar_full_s.slice(start_idx, cny_train.end_time())

cov_train_sgd = cov_train_sgd.astype(np.float32)
cov_train_cny = cov_train_cny.astype(np.float32)

cov_full = calendar_full_s

past_cov_sgd_train = cny_train_s
past_cov_sgd_full = cny_train_s.append(cny_val_s)

past_cov_cny_train = sgd_train_s
past_cov_cny_full = sgd_train_s.append(sgd_val_s)

tft = TFTModel(
    input_chunk_length=32,
    output_chunk_length=7,
    hidden_size=64,
    lstm_layers=1,
    num_attention_heads=4,
    dropout=0.1,
    batch_size=32,
    n_epochs=50,
    add_relative_index=True,
    likelihood=QuantileRegression(quantiles=[0.05, 0.5, 0.95]),
    random_state=42
)

tft.fit(
    series=[sgd_train_s, cny_train_s],
    past_covariates=[past_cov_sgd_train, past_cov_cny_train],
    future_covariates=[cov_train_sgd, cov_train_cny]
)

sgd_pred_scaled = tft.predict(
    n=h_sgd,
    series=sgd_train_s,
    past_covariates=past_cov_sgd_full,
    future_covariates=cov_full,
    show_warnings=False
)
sgd_pred = scaler_sgd.inverse_transform(sgd_pred_scaled)

cny_pred_scaled = tft.predict(
    n=h_cny,
    series=cny_train_s,
    past_covariates=past_cov_cny_full,
    future_covariates=cov_full,
    show_warnings=False
)
cny_pred = scaler_cny.inverse_transform(cny_pred_scaled)

print("TFT RMSE (SGD/USD):", rmse(sgd_val, sgd_pred))
print("TFT MAE  (SGD/USD):", mae(sgd_val, sgd_pred))
print("TFT MAPE (SGD/USD):", mape(sgd_val, sgd_pred))

print("\nTFT RMSE (CNY/USD):", rmse(cny_val, cny_pred))
print("TFT MAE  (CNY/USD):", mae(cny_val, cny_pred))
print("TFT MAPE (CNY/USD):", mape(cny_val, cny_pred))

plt.figure(figsize=(12, 5))
sgd_series.plot(label="Actual SGD/USD")
sgd_pred.plot(label="TFT forecast SGD/USD")
plt.title("SGD/USD — TFT Forecast")
plt.legend(); plt.show()

plt.figure(figsize=(12, 5))
cny_series.plot(label="Actual CNY/USD")
cny_pred.plot(label="TFT forecast CNY/USD")
plt.title("CNY/USD — TFT Forecast")
plt.legend(); plt.show()

**Interpretation**

What the plots show

- **Systematic upward bias** in the validation window for **both pairs**: the TFT forecasts (blue) tend to sit above the realized series (black).

- **SGD/USD**: the forecast spends most of the window around **~1.45–1.55**, while the actual series trades **~1.33–1.42**. The model captures broad direction at times, but it **overestimates the level** and smooths away short swings.

- CNY/USD: forecasts cluster roughly **~6.8–7.1** vs. actual **~6.6–7.0**. Again, there’s a **level bias upward** and the forecast path is too smooth, **missing faster dips/rallies**.

## Scale and Split Calendar Covariates

In [ ]:
scaler_covs = DartsScaler()
calendar_covs_s = scaler_covs.fit_transform(calendar_covs)

cov_train, cov_val = calendar_covs_s.split_after(sgd_train.end_time())

cov_full = cov_train.append(cov_val)

print("Covariates prepared and split successfully.")
print("cov_train shape:", cov_train.shape)
print("cov_val shape:", cov_val.shape)
print("cov_full shape:", cov_full.shape)

## EXGBoost Hypertunning

In [ ]:
def _build_xgb_from_params(params):
    p = {k: (int(v) if isinstance(v, (np.integer,)) else float(v) if isinstance(v, (np.floating,)) else v)
         for k, v in params.items()}
    lags = p.pop("lags", [-1, -2, -5])
    lags_past_covariates = p.pop("lags_past", [-1, -2, -5])
    lags_future = p.pop("lags_future", [0])
    out_chunk = p.pop("output_chunk_length", 1)

    allowed = {
        "n_estimators","learning_rate","max_depth","subsample","colsample_bytree",
        "min_child_weight","gamma","reg_alpha","reg_lambda","n_jobs","random_state"
    }
    xgb_kwargs = {k: v for k, v in p.items() if k in allowed}
    xgb_kwargs.setdefault("random_state", 42)
    xgb_kwargs.setdefault("n_jobs", -1)

    model = XGBModel(
        lags=lags,
        lags_past_covariates=lags_past_covariates,
        lags_future_covariates=lags_future,
        output_chunk_length=out_chunk,
        **xgb_kwargs,
    )
    used = {"lags": lags, "lags_past": lags_past_covariates, "lags_future": lags_future,
            "out_chunk": out_chunk, **xgb_kwargs}
    return model, used

def build_fx_features(ts: TimeSeries, prefix: str, max_window: int = 60) -> TimeSeries:

    s = pd.Series(ts.values().flatten(), index=ts.time_index).astype(float)

    ret1 = np.log(s / s.shift(1))
    ret5 = np.log(s / s.shift(5))

    ma7 = s.rolling(7).mean()
    ma14 = s.rolling(14).mean()
    ma30 = s.rolling(30).mean()
    std7 = s.rolling(7).std()
    std14 = s.rolling(14).std()

    feats = pd.DataFrame({
        f"{prefix}_ret1": ret1,
        f"{prefix}_ret5": ret5,
        f"{prefix}_ma7": ma7,
        f"{prefix}_ma14": ma14,
        f"{prefix}_ma30": ma30,
        f"{prefix}_std7": std7,
        f"{prefix}_std14": std14,
    }, index=s.index).dropna()

    return TimeSeries.from_times_and_values(
        times=feats.index, values=feats.values, columns=list(feats.columns)
    )

fe_sgd_full = build_fx_features(sgd_series, "sgd", max_window=60)
fe_cny_full = build_fx_features(cny_series, "cny", max_window=60)

full_past_cov_sgd = fe_sgd_full.stack(fe_cny_full)
full_past_cov_cny = fe_cny_full.stack(fe_sgd_full)

start = max(fe_sgd_full.start_time(), fe_cny_full.start_time())
end   = min(fe_sgd_full.end_time(), fe_cny_full.end_time())
fe_sgd_full = fe_sgd_full.slice(start, end)
fe_cny_full = fe_cny_full.slice(start, end)

sgd_train_start = max(sgd_train_s.start_time(), full_past_cov_sgd.start_time(), cov_train.start_time())
cny_train_start = max(cny_train_s.start_time(), full_past_cov_cny.start_time(), cov_train.start_time())

sgd_train_s_al  = sgd_train_s.slice(sgd_train_start, sgd_train_s.end_time())
cny_train_s_al  = cny_train_s.slice(cny_train_start, cny_train_s.end_time())

cov_train_sgd   = cov_train.slice(sgd_train_start, sgd_train_s.end_time())
cov_train_cny   = cov_train.slice(cny_train_start, cny_train_s.end_time())
past_cov_train_sgd = full_past_cov_sgd.slice(sgd_train_start, sgd_train_s.end_time())
past_cov_train_cny = full_past_cov_cny.slice(cny_train_start, cny_train_s.end_time())

params_sgd = {
    "lags": [-1, -2, -3, -5, -7, -10, -20, -40],
    "lags_past": [-1, -2, -5, -10, -20],
    "lags_future": [0,1,2,3,4,5,6],
    "output_chunk_length": 1,
    "n_estimators": 1000,
    "learning_rate": 0.03,
    "max_depth": 4,
    "min_child_weight": 6,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "reg_lambda": 1.0,
    "gamma": 0.0,
    "random_state": 42,
    "n_jobs": -1,
}
params_cny = {
    "lags": [-1, -2, -3, -5, -7, -10, -20, -40, -60],
    "lags_past": [-1, -2, -5, -10, -20, -40],
    "lags_future": [0,1,2,3,4,5,6],
    "output_chunk_length": 1,
    "n_estimators": 1200,
    "learning_rate": 0.02,
    "max_depth": 4,
    "min_child_weight": 8,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "reg_lambda": 1.5,
    "gamma": 0.0,
    "random_state": 42,
    "n_jobs": -1,
}

xgb_sgd, used_sgd = _build_xgb_from_params(params_sgd)
xgb_cny, used_cny = _build_xgb_from_params(params_cny)

print("[XGB_SGD] Using params:", used_sgd)

xgb_sgd.fit(
    series=sgd_train_s_al,
    past_covariates=past_cov_train_sgd,
    future_covariates=cov_train_sgd
)

H = len(sgd_val)

max_past_lag_sgd = abs(min(used_sgd['lags_past'])) if used_sgd['lags_past'] else 0
required_historical_start_time_sgd = sgd_val.start_time() - max_past_lag_sgd * sgd_val.freq

past_cov_pred_sgd = full_past_cov_sgd.slice(required_historical_start_time_sgd, sgd_val.end_time())

h_max = max(len(sgd_val), len(cny_val))
freq = pd.infer_freq(df.index) or "D"
offset = pd.tseries.frequencies.to_offset(freq)
max_future_lag = max(used_sgd['lags_future'] + used_cny['lags_future'])

future_cov_end_time = max(sgd_val.end_time(), cny_val.end_time()) + max_future_lag * offset

extended_future_cov_index = pd.date_range(
    start=cov_train.start_time(),
    end=future_cov_end_time,
    freq=freq
)

dow_ext = datetime_attribute_timeseries(extended_future_cov_index, attribute="day_of_week", one_hot=True)
mon_ext = datetime_attribute_timeseries(extended_future_cov_index, attribute="month", one_hot=True)
calendar_full_ext = dow_ext.stack(mon_ext)

calendar_full_ext_s = scaler_covs.transform(calendar_full_ext)

cov_full = calendar_full_ext_s

xgb_pred_s = xgb_sgd.predict(
    n=H,
    past_covariates=past_cov_pred_sgd,
    future_covariates=cov_full
)

pred_sgd = scaler_target.inverse_transform(xgb_pred_s)

print("\n[XGB_SGD] Validation metrics:")
print("  RMSE:", rmse(sgd_val, pred_sgd))
print("  MAE :", mae(sgd_val, pred_sgd))
print("  MAPE:", mape(sgd_val, pred_sgd))

plt.figure(figsize=(12, 6))
sgd_val.plot(label="Actual SGD/USD (Validation)")
pred_sgd.plot(label="XGB (feat) — SGD")
plt.title("SGD/USD — XGBoost with engineered covariates")
plt.legend(); plt.show()

print("\n[XGB_CNY] Using params:", used_cny)

xgb_cny.fit(
    series=cny_train_s_al,
    past_covariates=past_cov_train_cny,
    future_covariates=cov_train_cny
)

Hc = len(cny_val)

max_past_lag_cny = abs(min(used_cny['lags_past'])) if used_cny['lags_past'] else 0
required_historical_start_time_cny = cny_val.start_time() - max_past_lag_cny * cny_val.freq

past_cov_pred_cny = full_past_cov_cny.slice(required_historical_start_time_cny, cny_val.end_time())

pred_s_cny = xgb_cny.predict(
    n=Hc,
    past_covariates=past_cov_pred_cny,
    future_covariates=cov_full
)

pred_cny   = scaler_cny.inverse_transform(pred_s_cny)

print("\n[XGB_CNY] Validation metrics:")
print("  RMSE:", rmse(cny_val, pred_cny))
print("  MAE :", mae(cny_val, pred_cny))
print("  MAPE:", mape(cny_val, pred_cny))

plt.figure(figsize=(12, 6))
cny_val.plot(label="Actual CNY/USD (Validation)") # Plot only the validation actuals
pred_cny.plot(label="XGB (feat) — CNY")
plt.title("CNY/USD — XGBoost with engineered covariates")
plt.legend(); plt.show()

**Interpretation**

**SGD/USD**

- The blue forecast almost sits on top of the black actuals across the validation window.

- Turning points are captured with only a small lag during sharper moves (e.g., early-2018 dip and subsequent rebound).

- Amplitudes are well tracked; minor under/overshoot near local peaks is expected with autoregressive rolling prediction.

- Visual fit + very low MAPE (0.67%) suggest the feature set (lags, MAs, rolling vol; plus cross-pair info) is highly informative for SGD.

**CNY/USD**

- Direction and medium-term swings are captured; the model smooths the path a bit and underestimates peak amplitudes (e.g., late-2018/2019 run-ups).

- The forecast shows a slight lag around regime shifts; still, MAPE ~1.44% is strong for a level forecast on a managed FX pair.

**Why this works**

- Past-information features (lags, log-returns, moving averages, rolling volatility) give XGB immediate context about momentum and mean-reversion.

- Cross-pair covariates (SGD features helping CNY and vice-versa) capture co-movement in regional FX.

- Using 1-step autoregression (predicting next step repeatedly) leverages fresh information each step and helps track short-horizon dynamics.

**Remaining gaps / risks**

- Amplitude underestimation on CNY spikes, common with tree ensembles trained on squared error and smoothed features.

- Small forecast lag at turning points, intrinsic to autoregression and the chosen lag set.

- Feature leakage check: rolling statistics must be computed from past only.

- Single split can overstate performance; confirm with rolling backtests.

The **XGBoost** + engineered covariates approach is currently your best performer. Solidify it with rolling validation and targeted feature tweaks (especially for CNY) and consider returns-space modeling for further uplift.


## Save Models

In [ ]:
xgb_sgd.save("xgb_best_sgd.darts")
print("XGBoost model for SGD/USD saved as xgb_best_sgd.darts")

xgb_cny.save("xgb_best_cny.darts")
print("XGBoost model for CNY/USD saved as xgb_best_cny.darts")

## Load Models

In [ ]:
from darts.models import XGBModel

loaded_xgb_sgd = XGBModel.load("xgb_best_sgd.darts")
loaded_xgb_cny = XGBModel.load("xgb_best_cny.darts")

print("XGBoost models loaded successfully.")

## Conclusion & Business Interpretaions

**Executive summary**

- **Best performing approach: XGBoost with engineered covariates** (own lags, log-returns, moving averages, rolling volatility, plus cross-pair signals).

- **Accuracy (validation, daily horizon)**:

  - **SGD/USD**: RMSE **0.0127**, MAE **0.00925**, MAPE **0.67%**.

  - **CNY/USD**: RMSE **0.1206**, MAE **0.0968**, MAPE **1.44%**.

- Baselines & deep models: ARIMA(0,1,0)+drift and TFT/LSTM underperformed on this dataset/horizon; XGB materially reduced error (e.g., vs SGD ARIMA RMSE ~0.0328, XGB is ~2.6× better).

These results are strong enough for **short-horizon hedging support** and **rate-setting guidance**, especially for SGD/USD. CNY/USD is usable but benefits from regime/volatility features.

**KPI scorecard**

**1) Forecasting Accuracy (RMSE/MAE/APE)**

- **Pass (SGD)**: sub-1% MAPE; errors ~0.009 USD/SGD on average.

- **Good (CNY)**: ~1.4% MAPE; some amplitude underestimation around policy-driven moves.

**2) Trend Identification**

- **SGD**: Captures medium-term drifts and turning points with small lag.

- **CNY**: Directional trends are captured; peaks are smoothed—add regime features to sharpen.

**3) Volatility Prediction**

- Rolling-volatility features help the model respond to choppier periods; still, **sudden regime shifts** (e.g., policy adjustments) are partially smoothed. Recommend adding **breakout/vol-spike flags** and **shorter-window returns (1–3–5)**.

**4) Time-Horizon Performance**

- Excellent at **daily** horizon via 1-step autoregression.

- For **weekly/monthly** KPIs, aggregate daily forecasts and evaluate; consider a **multi-horizon direct** variant (e.g., output_chunk_length=5/20) for comparison.

**5) Risk Assessment (cost of error)**

- Translate MAE into exposure terms:

  - Example: with X million SGD exposure, expected P&L impact ≈ **MAE × exposure** (in quote currency). For SGD/USD, **0.00925 × exposure_SGD** gives expected absolute error in USD.

- Add **95% residual quantile** (or residual VaR) as a buffer when quoting or hedging; re-estimate after any model change.

**6) Business Interpretation / Actionability**

- **Hedging & reserves**: use daily forecasts + intervals to time hedge top-ups and size reserve buffers; prioritize SGD where accuracy is highest.

- **Pricing/Spreads**: set client spreads to **(execution costs + 95% error band)**; the band can be learned from rolling residuals.

- **Early-warning**: monitor residual spikes; when residuals breach control limits, flag **regime change** and trigger re-fit or human review.

- **Attribution**: XGB feature importance/SHAP explains which signals (e.g., short-term momentum, rolling vol) drove each forecast—useful for desk commentary.

**Model risks & controls**

- **Regime risk (CNY)**: Managed currency & policy shocks → add **regime indicators**, **macro proxies** (rate differentials, DXY), and **event dummies**.

- **Data leakage**: keep all rolling stats strictly **past-only** (you already drop look-ahead—keep guarding this).

- **Model drift**: schedule **monthly rolling backtests**; re-fit when RMSE/MAE degrade beyond thresholds.

- **Champion/Challenger**: deploy XGB as **champion**; keep **ARIMA RW-drift** and a **returns-space XGB** as challengers.

**Recommended next steps:**

- **Walk-forward backtesting**: 6–12M rolling windows, step monthly; report mean/median RMSE/MAE vs baselines.

- **CNY uplift**: add short-horizon returns (1/3/5), breakout flags, rate differential/DXY proxies; try slightly deeper trees (max_depth 5) with stronger regularization (min_child_weight, reg_lambda).

- **Returns space modeling**: forecast log-returns, then reconstruct price; often improves turning-point response.

- **Uncertainty bands**: residual bootstrap or quantile XGB (pinball loss) fo8r 5–95% bands.

- **MLOps**: retrain monthly, drift alarms on residuals, model registry with versioning, and Champion/Challenger governance.



### Deployment recommendation

**Streamlit suggested app layout**:

- **Sidebar**: currency pair, horizon (days), model (XGB, ARIMA), feature toggles.

- **Main panels**:

  - **Forecast chart** (actual vs forecast; optional prediction band from residual bootstrap).

  - **Metrics & KPI tiles** (RMSE/MAE/MAPE; rolling backtest summary).

  - **Feature importance / SHAP** (top drivers for current forecast window).

  - **Risk view** (95% residual band, estimated P&L impact for a user-entered exposure).

  - **Download** (CSV of forecast + intervals).

**When to prefer alternatives**:

- **Production integration / low latency**: expose a **FastAPI** microservice for forecasts; let downstream systems (pricing/hedging) consume JSON.

- **Enterprise dashboards: Dash or Power BI** if IT standardizes on them.

- **Scheduling & monitoring**: orchestrate daily runs via **Airflow/Prefect**, push metrics to **Prometheus/Grafana**.

### Final Analysis

- The **XGBoost + engineered covariates** solution meets the accuracy KPI (especially for **SGD/USD**) and is operationally actionable for hedging and pricing decisions.

- With modest feature/regularization tweaks, **CNY/USD** can be tightened further.

- Package the model in a **Streamlit app** for desks to explore forecasts and risk quickly, and back it with a **FastAPI** endpoint for system integration.